# GENIE Learning Scenarios with Carbon Taxes [TEST]

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

from message_ix.tools.add_learning import add_learning


%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform("local", jvmargs=['-Xmx8G'])

## Making a clone of the existing scenario 'baseline'

Dump scenario to excel nad create a scenario from excel

In [3]:
base = message_ix.Scenario(
    mp, model="GENIE", scenario="learning",
    )

Clone baseline scenario and add learning parameters

## High CO2 Tax Scenarios

Here, we run the model under a range of CO2 tax scenaros

In [4]:
# Firstly, let's create an empty list to contain the emissions data
tax_growth = {'n':0,'l':0.025,'m':0.050,'h':0.075}
data_collect = {par: [] for par in list(tax_growth.keys())}


In [5]:
# Scenario setup
txs = 'n'
lrn = 'y'
rnd = 'n'

Base2020 = 0 if txs == 'n' else 10 # $/tCO2
lrn_val = 1 if lrn == 'y' else 0

In [6]:
scen = base.clone(
    "GENIE",
    "with_"+lrn+"learning_CT"+txs+"-RNDn",
    txs+" carbon tax with"+rnd+"RND target",
    keep_solution=False,
    )
scen.check_out()

all_years = scen.set('year')
all_nodes = scen.set('node')
type_years = list(all_years[all_years>=2020])
nodes = list(all_nodes[all_nodes=="World"])

growth = tax_growth.get(txs) # lo: 3%, med: 5%, hi: 10

value_taxes = [round((Base2020*((1+growth)**(x-2020))),2) for x in type_years]


Add CO2 Tax to the scenario

In [7]:
# In case you need to add the new unit to the platform
mp.add_unit("USD/tCO2")
tax_emission = []
for n in nodes:
    data = make_df(
        "tax_emission",
        node=n,
        type_year=type_years,
        type_tec="all",
        unit="USD/tCO2",
        type_emission="TCE",
        value=value_taxes,
        )
    tax_emission.append(data)

tax_emission = pd.concat(tax_emission)

scen.add_par("tax_emission", tax_emission)

tax_emission

,node,type_emission,type_tec,type_year,value,unit
0,World,TCE,all,2020,0,USD/tCO2
1,World,TCE,all,2025,0,USD/tCO2
2,World,TCE,all,2030,0,USD/tCO2
3,World,TCE,all,2035,0,USD/tCO2
4,World,TCE,all,2040,0,USD/tCO2
5,World,TCE,all,2045,0,USD/tCO2
6,World,TCE,all,2050,0,USD/tCO2
7,World,TCE,all,2055,0,USD/tCO2
8,World,TCE,all,2060,0,USD/tCO2
9,World,TCE,all,2070,0,USD/tCO2


In [8]:
scen.commit(comment="GENIE with"+lrn+"learning |"+txs+"carbon tax |"+rnd+"RND target")

In [9]:
scen.solve(gams_args =["--learningmode="+str(lrn_val)],solve_options={"scaind":-1})

In [10]:
scen.var("OBJ")["lvl"]

3115606.75

In [14]:
techs = ['coal_adv','coal_adv_ccs','coal_ppl','coal_ppl_u',
         'igcc','igcc_ccs','gas_cc','gas_cc_ccs','gas_ct',
         'gas_htfc','gas_ppl','bio_istig','bio_istig_ccs',
         'bio_ppl','geo_ppl','hydro_hc','hydro_lc','nuc_fbr',
         'nuc_hc','nuc_lc','foil_ppl','loil_cc','lio_ppl','oil_ppl',
         'SO2_scrub_ppl','solar_pv_ppl','csp_sm1_ppl','csp_sm3_ppl',
         'wind_ppf','wind_ppl','h2_bio','h2_bio_ccs','eth_bio',
         'eth_bio_ccs','liq_bio','liq_bio_ccs']
capacity = (scen.var("CAP", filters={'year_act':2050,'technology':techs})
            .drop(columns='mrg').groupby('technology').sum().reindex(techs))
capacity.to_excel('output.xlsx', index=True)

C:\Users\pratama\AppData\Local\Temp\ipykernel_9364\1306185295.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .drop(columns='mrg').groupby('technology').sum().reindex(techs))


In [16]:
scen.var("EMISS", filters={'node':'World','type_tec':'all','emission':'TCE','year':2050})

,node,emission,type_tec,year,lvl,mrg
0,World,TCE,all,2050,19505.405214,0.0


## Close the connection to the database

In [13]:
mp.close_db()